**Reproducing GAP results (Code repository: https://github.com/sattree/gpr_pub)**
===

A best-effort attempt to reproduce the results presented in the research paper - [Mind the GAP: A Balanced Corpus of Gendered Ambiguous Pronouns](https://arxiv.org/pdf/1810.05201.pdf)
***

This kernel is the second in a tri-series of self-contained installments to introduce the GPR problem.
1. Coref visualization - https://www.kaggle.com/sattree/1-coref-visualization-jupyter-allenlp-stanford
1. **Reproducing GAP results**
1. A better baseline - without any training

The notebook only presents results and demonstrates model usage to avoid flooding it with code statements. The model codes are available at https://github.com/sattree/gpr_pub. Hope you will find the code structure easily navigable.
***
**Executive Summary**
---

First, an important note about the results presented in the paper.

The paper uses a different evaluation strategy than the one being used in kaggle. The evaluation metric in the paper assumes predictions for mention A and B to be conditionally independent, while in reality they are mutually exclusive. As a consequence of this, any model performing better than a random guess would enjoy an upward drift in measured performance. Fortunately, this has been fixed in the kaggle multi-class formulation of the problem, and models will not be able to leverage such unfair rewards during evaluation.

Moving on, below is a summary of the results and findings:

<table>
    <tr>
        <td><img src="https://github.com/sattree/gpr_pub/blob/master/docs/gap_results.png?raw=true" alt="GAP results" style="width: 45%;"/> </td>
        <td> <img src="https://github.com/sattree/gpr_pub/blob/master/docs/gap_results_reproduced.png?raw=true" alt="Reproduced GAP results" style="width: 55%;"/>
                <div style="text-align:center; margin-top: 8px;">Table: Results reproduced by this kernel - predictions scored by gap_scorer.py script provided by the GAP authors in their code repository.</div>
        </td>
    </tr>
    <tr>
         <td>
             <img src="https://github.com/sattree/gpr_pub/blob/master/docs/results_kaggle_formulation.png?raw=true" alt="Reproduced GAP results with kaggle formulation" style="width: 60%;"/>
             <div style="text-align:center; margin-top: 8px;">Table: Results produced by Parallelism+URL heuristic with appropriate fallbacks and evaluated according to multi-class kaggle formulation</div>
        </td>
        <td>
            <div style="text-align:center; margin-top: 8px;"><strong>0.8393752431386904</strong></div>
            <div style="text-align:center; margin-top: 8px;">Log loss of the predictions generated by Parallelism+URL heuristic with appropriate fallbacks</div>
        </td>
    </tr>
</table>

**Heuristic Models Reproduced**
* Surface cues
    * Random
    * Token Distance
* Structural cues
    * Syntactical Distance
    * Parallelism
* Wikipedia cues
    * URL

**Results Discussion**
1. Results were reproduced for gold-two-mention task alone, since it is directly applicable to this competition.
1. Looking at the first two figures, it appears that all the models have been reproduced well. The numbers are quite close other than for Random Model (explained below).
1. Only heuristics based results (showcased by the authors as best performing in their study) were reproduced. The results from pre-trained models will be presented in the upcoming kernel - 'A better baseline'.
1. Topical Entity results were not reproduced - for gold-two-mentions the frequency of each token will almost always be 1. This should lead to the same results as 'Token Distance' heuristic with a small variance.
1. Random Model - it is suprising that the author's random model generated a performance close to 50% given that 10% of the samples actually belong to 'NEITHER' category.
1. Kaggle multi class evaluation - the discrepancy in evaluation is evident from figures 2 and 3 (72 vs 65) and illustrates the argument made above about the difference in the two evaluation strategies.
1. Log loss evaluation of the proposed heuristics measured by kaggle's metric of choice - the observed performance indicates that the heuristics models serve as a pretty strong baseline for this competition. You may have noticed (in the code below) an adjustment of 0.25 to the predictions before the calculation of log loss. Log loss is applied to the probabilities of predictions whereas we only have access to hard labels. Log loss will induce a non linear penalty and will penalize the predictions exponentially as they drift away from ground truth. To avoid extreme penalties, I use a crude lower bound guesstimate to bound the predictions away from zero. 

**Further Improvements**

There can be two straighforward extensions to further improve the performance of the heuristic models:
1. A confidence model - to map the the predicted labels to probabilities. This should provide a significant bump to log loss evaluation.
1. Replace token distance model as backoff - the token distance model enforces a decision for  'NEITHER' category as 'A' or 'B', which actually constitutes a significant portion (10%) of the samples, and necessarily accepts mispredictions on a subset of these 10% (those filtering down to this level in the stack of models). This may also be handled by the same confidence model as mentioned above.

***

In [ ]:
# Download and install all dependencies
# gpr_pub contains the heuristics models and supplementary code
!git clone https://github.com/sattree/gpr_pub.git
!wget http://nlp.stanford.edu/software/stanford-corenlp-full-2018-10-05.zip
!unzip stanford-corenlp-full-2018-10-05.zip
!pip install allennlp --ignore-installed greenlet
!pip install attrdict

In [ ]:
import pandas as pd
from sklearn.metrics import log_loss, classification_report
from attrdict import AttrDict

import spacy

from allennlp.predictors.predictor import Predictor
from allennlp.models.archival import load_archive
from nltk.parse.corenlp import CoreNLPParser, CoreNLPDependencyParser

from gpr_pub.utils import CoreNLPServer

# gap_scorer_ext has minor fixes for py3 and to take pandas df as input instead of filepaths
from gpr_pub.gap.gap_scorer_ext import read_annotations, calculate_scores, add_to_score_view

* gpr_pub.models.heuristics
    - Contains all the heuristic models, the code repository contains models in addition to the ones being presented here.
* gpr_pub.models.pronoun_resolution
    - Resolves pronoun mentions from the clusters predicted by the coref models
* gpr_pub.models.multi_pass_sieve
    - Implements Backoff mechanism
    - The functionality is similar to the multi-pass sieve algorithm proposed for coref resolution by manning et al.
    - The stack of models is applied in the order of decreasing precision from top to bottom (or left to right if seen as arguments to the sieve model)
    - Leads to increasing recall in the same direction

In [ ]:
# Heuristic models implement coref resolution based on heuristics described in the paper
# Pronoun resolution is a simple wrapper to convert coref predictions into class-specific labels
# Multi pass sieve model implements backoff mechanism
from gpr_pub.models.heuristics.random_distance import RandomModel
from gpr_pub.models.heuristics.token_distance import TokenDistanceModel
from gpr_pub.models.heuristics.syntactic_distance import StanfordSyntacticDistanceModel
from gpr_pub.models.heuristics.parallelism import AllenNLPParallelismModel as ParallelismModel
from gpr_pub.models.heuristics.url_title import StanfordURLTitleModel as URLModel

from gpr_pub.models.pronoun_resolution import PronounResolutionModel

from gpr_pub.models.multi_pass_sieve import MultiPassSieveModel

In [ ]:
# Instantiate stanford corenlp server
STANFORD_CORENLP_PATH = 'stanford-corenlp-full-2018-10-05/'
server = CoreNLPServer(classpath=STANFORD_CORENLP_PATH,
                        corenlp_options=AttrDict({'port': 9090, 
                                                  'timeout': '600000', 
                                                  'quiet': 'true',
                                                  'preload': 'tokenize,spplit,lemma,parse,deparse'}))
server.start()
STANFORD_SERVER_URL = server.url

In [ ]:
# Instantiate base models
STANFORD_MODEL = CoreNLPParser(url=STANFORD_SERVER_URL)
SPACY_MODEL = spacy.load('en_core_web_lg')
model_url = 'https://s3-us-west-2.amazonaws.com/allennlp/models/biaffine-dependency-parser-ptb-2018.08.23.tar.gz'
archive = load_archive(model_url, cuda_device=0)
ALLEN_DEP_MODEL = Predictor.from_archive(archive)

In [ ]:
# Load data
train = pd.read_csv('gpr_pub/data/gap-development.tsv', sep='\t')
# normalizing column names
train.columns = map(lambda x: x.lower().replace('-', '_'), train.columns)
with pd.option_context('display.max_rows', 10, 'display.max_colwidth', 15):
    display(train)

In [ ]:
# Instantiate heuristic models
random_coref_model = RandomModel(SPACY_MODEL)
random_proref_model = PronounResolutionModel(random_coref_model)

token_distance_coref_model = TokenDistanceModel(SPACY_MODEL)
token_distance_proref_model = PronounResolutionModel(token_distance_coref_model)

syntactic_distance_coref_model = StanfordSyntacticDistanceModel(STANFORD_MODEL)
syntactic_distance_proref_model = PronounResolutionModel(syntactic_distance_coref_model, n_jobs=-1)

parallelism_coref_model = ParallelismModel(ALLEN_DEP_MODEL, SPACY_MODEL)
parallelism_proref_model = PronounResolutionModel(parallelism_coref_model)

url_title_coref_model = URLModel(STANFORD_MODEL)
url_title_proref_model = PronounResolutionModel(url_title_coref_model, n_jobs=-1)

In [ ]:
# Creates sieve pipeline of heuristic models, applying each new heuristic with appropriate backoff models
# Multi pass sieve - order of models provided as input is important
#    - left to right: recall increases
#    - right to left: precision increases
preds = MultiPassSieveModel(random_proref_model).predict(train)
score_df = add_to_score_view(preds, train, None, 'Random')

preds = MultiPassSieveModel(token_distance_proref_model).predict(train)
score_df = add_to_score_view(preds, train, score_df, 'Token Distance')

preds = MultiPassSieveModel(syntactic_distance_proref_model,
                           token_distance_proref_model).predict(train)
score_df = add_to_score_view(preds, train, score_df, 'Syntactic Distance')

preds = MultiPassSieveModel(parallelism_proref_model,
                            syntactic_distance_proref_model,
                           token_distance_proref_model).predict(train)
score_df = add_to_score_view(preds, train, score_df, 'Parallelism')

preds = MultiPassSieveModel(url_title_proref_model,
                            parallelism_proref_model,
                            syntactic_distance_proref_model,
                           token_distance_proref_model).predict(train)

In [ ]:
score_df = add_to_score_view(preds, train, score_df, 'Parallelism+URL')

Evaluate the predictions as per kaggle fromulation for comparison
---

In [ ]:
y_pred = pd.DataFrame(preds, columns=['A', 'B'])
y_pred['NEITHER'] = ~y_pred['A'] & ~y_pred['B']

y_true = pd.DataFrame(train[['a_coref', 'b_coref']].values, columns=['A', 'B'])
y_true['NEITHER'] = ~y_true['A'] & ~y_true['B']

print(classification_report(y_true, y_pred, target_names=['A', 'B', 'NEITHER']))

In [ ]:
# shift the predictions by 0.25
log_loss(y_true, y_pred + 0.25)

Generate predictions for submission
---

In [ ]:
test_df = pd.read_csv('../input/test_stage_1.tsv', sep='\t')
# normalizing column names
test_df.columns = map(lambda x: x.lower().replace('-', '_'), test_df.columns)
preds = MultiPassSieveModel(url_title_proref_model,
                            parallelism_proref_model,
                            syntactic_distance_proref_model,
                           token_distance_proref_model).predict(test_df)
preds = pd.DataFrame(preds, columns=['A', 'B'])

sub_df = pd.read_csv('../input/sample_submission_stage_1.csv')
sub_df.loc[:, 'A'] = preds['A'] + 0.25
sub_df.loc[:, 'B'] = preds['B'] + 0.25
sub_df.loc[:, 'NEITHER'] = (~preds['A'] & ~preds['B']) + 0.25

sub_df.to_csv("submission.csv", index=False)

In [ ]:
sub_df.head()

In [ ]:
!rm -r stanford-corenlp-full-2018-10-05/
!rm -r gpr_pub/
!rm stanford-corenlp-full-2018-10-05.zip

This concludes the second installment.
Hope you find the kernel and models presented useful!

Don't forget to check out  Coref visualization - https://www.kaggle.com/sattree/1-coref-visualization-jupyter-allenlp-stanford!

**Stay tuned for the last and final episode - A better baseline!**
